In [ ]:
python -m pip install g

In [1]:
from flask import Flask, request, jsonify
import os
import base64
from github import Github

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'  # Temporary folder to store uploaded files

# Ensure the upload folder exists
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:
        file_path = os.path.join(UPLOAD_FOLDER, file.filename)
        file.save(file_path)

        # Upload to GitHub
        upload_to_github(file_path, file.filename)

        return jsonify({'message': 'File uploaded successfully!'}), 200

def upload_to_github(file_path, filename):
    # GitHub credentials and repository details
    token = 'your_github_token'
    repo_name = 'your_github_username/your_repo_name'
    commit_message = f'Add {filename}'

    # Read the file content
    with open(file_path, 'rb') as file:
        content = file.read()

    # Initialize GitHub client
    g = Github(token)
    repo = g.get_repo(repo_name)

    # Create or update the file in the repository
    try:
        # Check if the file already exists
        contents = repo.get_contents(filename)
        repo.update_file(contents.path, commit_message, content, contents.sha)
    except:
        # If file does not exist, create it
        repo.create_file(filename, commit_message, content)

    # Remove the file from the local upload folder
    os.remove(file_path)

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'github'